This is a very brief demo to show how one could go from an environment with just a Python 3 runtime to pyhf installed and being useful.

## Installation

To take advantage of the great tools that `uproot` provides us let's install `pyhf` with the [`xmlio` options](https://diana-hep.org/pyhf/examples/notebooks/XML_ImportExport.html)

```
$ python3 -m pip install pyhf[xmlio]
```

but this is Binder! so I've already done that for you. :)

In [2]:
! pyhf --version

pyhf, version 0.2.2


## Running with CLI

Now that we have `pyhf` installed let's ask what the CLI can do for us

In [1]:
! pyhf --help

Usage: pyhf [OPTIONS] COMMAND [ARGS]...

Options:
  --version   Show the version and exit.
  -h, --help  Show this message and exit.

Commands:
  cls
  inspect
  json2xml
  xml2json  Entrypoint XML: The top-level XML file for the PDF definition.


and then let's use `pyhf inspect` to check to see if our `example.json` is in a correct form for the schema

In [2]:
! pyhf inspect example.json

          Summary       
    ------------------  
       channels  1
        samples  2
     parameters  2
      modifiers  2

       channels  nbins
     ----------  -----
  singlechannel    2  

        samples
     ----------
     background
         signal

     parameters  constraint              modifiers
     ----------  ----------              ----------
             mu  unconstrained           normfactor
uncorr_bkguncrt  constrained_by_poisson  shapesys

    measurement           poi            parameters
     ----------        ----------        ----------
(*) Measurement            mu            (none)



finally use `pyhf cls` to get the observed CL_s and the expected CL_s band values (the "[Brazil Band](https://arxiv.org/pdf/1306.3117.pdf#page=102)")

In [3]:
! pyhf cls example.json

{
    "CLs_exp": [
        0.0004090387453250841,
        0.0032606968023913925,
        0.02255257597653917,
        0.11898700005707148,
        0.39844667251932997
    ],
    "CLs_obs": 0.053994246621274014
}


### Expanding the CLI with other tools

Additionally, you can also use [JSON Patch](https://tools.ietf.org/html/rfc6902) to patch a new signal in and update the likelihood (think reinterpretation).

In [4]:
! pyhf cls example.json --patch new_signal.json

{
    "CLs_exp": [
        0.031409129113842485,
        0.09019050672644717,
        0.2345563523518709,
        0.5053369568952392,
        0.8098259293224213
    ],
    "CLs_obs": 0.3536906623262466
}


and as the return format from `pyhf cls` is JSON you can also use [`jq`](https://stedolan.github.io/jq/) to filter results

In [5]:
! pyhf cls example.json --patch new_signal.json | jq .CLs_obs

0.3536906623262466


and of course given the way the CLI is setup you can pipe in the JSON which means that you can grab remotely hosted files (think HEPData). Here we `curl` down the Background only and signal patch JSON from HEPData

```
$ curl -sL https://doi.org/10.17182/hepdata.89408.v1/r2 | \
  tar -O -xzv RegionA/BkgOnly.json | \
  pyhf cls --patch <(curl -sL https://doi.org/10.17182/hepdata.89408.v1/r2 | \
    tar -O -xzv RegionA/patch.sbottom_1300_205_60.json) | \
  jq .CLs_obs
```

In [6]:
! curl -sL https://doi.org/10.17182/hepdata.89408.v1/r2 | tar -O -xzv RegionA/BkgOnly.json > RegionA_BkgOnly.json
! curl -sL https://doi.org/10.17182/hepdata.89408.v1/r2 | tar -O -xzv RegionA/patch.sbottom_1300_205_60.json > RegionA_patch.sbottom_1300_205_60.json
! pyhf cls RegionA_BkgOnly.json --patch RegionA_patch.sbottom_1300_205_60.json | jq .CLs_obs

RegionA/BkgOnly.json
RegionA/patch.sbottom_1300_205_60.json
/home/jovyan/src/pyhf/src/pyhf/tensor/numpy_backend.py:252: RuntimeWarning: invalid value encountered in log
  return n * np.log(lam) - lam - gammaln(n + 1.0)
0.24443635754482018


and compare to another signal point

```
$ curl -sL https://doi.org/10.17182/hepdata.89408.v1/r2 | \
  tar -O -xzv RegionA/BkgOnly.json | \
  pyhf cls --patch <(curl -sL https://doi.org/10.17182/hepdata.89408.v1/r2 | \
    tar -O -xzv RegionA/patch.sbottom_1300_230_100.json) | \
  jq .CLs_obs

```

In [7]:
! curl -sL https://doi.org/10.17182/hepdata.89408.v1/r2 | tar -O -xzv RegionA/patch.sbottom_1300_230_100.json > RegionA_patch.sbottom_1300_230_100.json
! pyhf cls RegionA_BkgOnly.json --patch RegionA_patch.sbottom_1300_230_100.json | jq .CLs_obs

RegionA/patch.sbottom_1300_230_100.json
/home/jovyan/src/pyhf/src/pyhf/tensor/numpy_backend.py:252: RuntimeWarning: invalid value encountered in log
  return n * np.log(lam) - lam - gammaln(n + 1.0)
0.040766025813435774


Those URLs are hard to read (Jupyter shell magics are only so magic), so the following is happening:

```
curl -sL https://web.end.point/model.json \
  | pyhf cls --patch <(curl -sL https://web.end.point/new_signal.json)
```